# Importing Dependencies

In [ ]:
# Pip install ultralytics and dependencies and check software and hardware.
%pip install ultralytics -q
import ultralytics
import os
import cv2
from ultralytics import YOLO
ultralytics.checks()

In [ ]:
VIDEOS_DIR = 'sample.mp4'
video_path='sample.mp4'

In [ ]:
cap = cv2.VideoCapture(video_path)

In [ ]:
ret, frame = cap.read()

In [ ]:
MODEL_DIR = 'license_plate_detector.pt'
model_path = 'license_plate_detector.pt'

In [ ]:
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# Load YOLO model
MODEL_PATH = "license_plate_detector.pt"  # Replace with your YOLO model path
model = YOLO(MODEL_PATH)

# Video input and output
video_path = "input_video.mp4"  # Replace with your input video file path
output_path = "output_video.mp4"  # Output video file path

cap = cv2.VideoCapture(video_path)

# Check if the video is opened successfully
if not cap.isOpened():
    print(f"Error: Cannot open video {video_path}")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Detection threshold
threshold = 0.5

# Read and process video frames
ret, frame = cap.read()
while ret:
    # Perform detection
    results = model(frame)[0]

    # Annotate detections on the frame
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        if score > threshold:
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            # Add label
            cv2.putText(frame, f"{model.names[int(class_id)]} {score:.2f}", (int(x1), int(y1) - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Display frame using matplotlib
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show()

    # Write processed frame to output video
    out.write(frame)

    # Read the next frame
    ret, frame = cap.read()

# Release resources
cap.release()
out.release()
print(f"Processed video saved to {output_path}")


In [ ]:
import cv2
import matplotlib.pyplot as plt
from ultralytics import YOLO

# Load YOLO model
model_path = "license_plate_detector.pt"
model = YOLO(model_path)

# Input and output video paths
video_path = 'sample.mp4'
output_path = 'sample_output.mp4'

# Initialize video capture
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define VideoWriter
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Detection threshold
threshold = 0.5

# Initialize frame counter for debugging
frame_count = 0

# Process video frames
ret, frame = cap.read()
while ret:
    # Perform inference
    results = model(frame)[0]

    # Annotate detections on the frame
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result
        if score > threshold:
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 4)
            label = f"{results.names[int(class_id)].upper()} {score:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1 - 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Save the processed frame to the output video
    out.write(frame)

    # Optional: Save the frame as an image for debugging
    cv2.imwrite(f"frame_{frame_count}.jpg", frame)
    frame_count += 1

    # Optional: Display the frame using matplotlib (alternative to cv2.imshow)
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.show(block=False)
    plt.pause(0.001)
    plt.close()

    # Read next frame
    ret, frame = cap.read()

# Clean up
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"Processed video saved to {output_path}")


In [ ]:
import cv2
import os
from ultralytics import YOLO  # or another YOLO module, adjust accordingly

# Load the YOLO model
model_path = "license_plate_detector.pt"  # Replace with your model path
model = YOLO(model_path)  # Load the custom model

# Initialize the video capture and video writer
video_path = 'sample.mp4'  # Input video path
output_path = 'sample_output.mp4'  # Output video path
cropped_frames_dir = "cropped_frames"  # Directory to save cropped images
os.makedirs(cropped_frames_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)

# Check if the video was successfully opened
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties (width, height, fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Threshold for object detection
threshold = 0.5
padding = 20  # Pixels to expand around the bounding box

frame_count = 0  # Keep track of frame numbers

# Read and process frames from the video
ret, frame = cap.read()
while ret:
    frame_count += 1

    # Perform inference
    results = model(frame)[0]

    # Draw boxes and labels on the frame
    for result in results.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = result

        if score > threshold:
            # Adjust the bounding box to include padding
            x1 = max(0, int(x1) - padding)
            y1 = max(0, int(y1) - padding)
            x2 = min(frame_width, int(x2) + padding)
            y2 = min(frame_height, int(y2) + padding)

            # Draw enlarged bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)

            # Save the cropped region (enlarged area around license plate)
            cropped_frame = frame[y1:y2, x1:x2]
            cropped_path = f"{cropped_frames_dir}/cropped_frame_{frame_count}.jpg"
            cv2.imwrite(cropped_path, cropped_frame)

            # Put label text
            label = f"{results.names[int(class_id)].upper()} {score:.2f}"
            cv2.putText(frame, label, (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

    # Write the processed frame to the output video
    out.write(frame)

    # Read next frame
    ret, frame = cap.read()

# Release video capture and writer objects
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Cropped frames saved in: {cropped_frames_dir}")


In [ ]:
from flask import Flask, render_template, request, send_file, redirect, url_for
import os
import cv2
from ultralytics import YOLO
import zipfile

# Initialize Flask app
app = Flask(__name__)

# Paths
UPLOAD_FOLDER = 'uploads'
PROCESSED_FOLDER = 'processed'
CROPPED_FOLDER = 'processed/cropped_frames'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(PROCESSED_FOLDER, exist_ok=True)
os.makedirs(CROPPED_FOLDER, exist_ok=True)

# YOLO model
MODEL_PATH = "license_plate_detector.pt"  # Replace with your YOLO model path
model = YOLO(MODEL_PATH)

@app.route('/')
def home():
    return render_template('index.html', section="upload")

@app.route('/process', methods=['POST'])
def process_video():
    if 'video' not in request.files:
        return redirect(url_for('home'))

    video = request.files['video']
    if video.filename == '':
        return redirect(url_for('home'))

    # Save uploaded video
    video_path = os.path.join(UPLOAD_FOLDER, video.filename)
    video.save(video_path)

    # Process video
    processed_video_path = os.path.join(PROCESSED_FOLDER, 'processed_' + video.filename)
    process_license_plates(video_path, processed_video_path)

    # Create ZIP file of cropped frames
    zip_path = os.path.join(PROCESSED_FOLDER, 'cropped_frames.zip')
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for root, _, files in os.walk(CROPPED_FOLDER):
            for file in files:
                zipf.write(os.path.join(root, file), arcname=file)

    return render_template('index.html', section="result", video_filename=os.path.basename(processed_video_path), zip_filename='cropped_frames.zip')

@app.route('/download/<filename>')
def download_file(filename):
    file_path = os.path.join(PROCESSED_FOLDER, filename)
    return send_file(file_path, as_attachment=True)

def process_license_plates(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Error: Could not open video file {video_path}.")

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Define VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    threshold = 0.5
    padding = 20
    frame_count = 0

    ret, frame = cap.read()
    while ret:
        frame_count += 1

        results = model(frame)[0]

        for result in results.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = result
            if score > threshold:
                x1 = max(0, int(x1) - padding)
                y1 = max(0, int(y1) - padding)
                x2 = min(frame_width, int(x2) + padding)
                y2 = min(frame_height, int(y2) + padding)

                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 4)
                label = f"{model.names[int(class_id)].upper()} {score:.2f}"
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 255, 0), 3, cv2.LINE_AA)

                # Save cropped frames
                cropped_frame = frame[y1:y2, x1:x2]
                cropped_path = os.path.join(CROPPED_FOLDER, f"cropped_frame_{frame_count}.jpg")
                cv2.imwrite(cropped_path, cropped_frame)

        out.write(frame)
        ret, frame = cap.read()

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    app.run()
